In [3]:
using Pkg
pkg_folder = "../"
Pkg.activate(pkg_folder)
using BindingAndCatalysis

  Activating project at `~/Realizibility_index/BindingAndCatalysis.jl`


In [4]:
N = [1 0 0 0 1 -1 0 0 0 0; 0 1 0 0 1 0 -1 0 0 0; 0 0 0 0 0 1 1 -1 0 0; 1 0 0 0 0 0 0 1 -1 0; 0 0 0 0 0 0 1 0 1 -1]

5×10 Matrix{Int64}:
 1  0  0  0  1  -1   0   0   0   0
 0  1  0  0  1   0  -1   0   0   0
 0  0  0  0  0   1   1  -1   0   0
 1  0  0  0  0   0   0   1  -1   0
 0  0  0  0  0   0   1   0   1  -1

In [5]:
# N = N_generator(5,10)
model = Bnc(N=N)

----------Binding Network Summary:-------------
Number of species (n): 10
Number of conserved quantities (d): 5
Number of reactions (r): 5
L matrix: [1 0 … 2 2; 0 1 … 1 2; … ; 0 0 … 0 0; 0 0 … 2 3]
N matrix: [1 0 … 0 0; 0 1 … 0 0; … ; 1 0 … -1 0; 0 0 … 1 -1]
Direction of binding reactions: backward
Catalysis involved: No
Regimes constructed: No
-----------------------------------------------

In [6]:
find_all_vertices!(model)
summary(model)

---------------------Start finding all vertices, it may takes a while.--------------------
Finished, with 79 vertices found and 51 asymptotic vertices.

-------------Start calculating nullity for each vertex, it also takes a while.------------
1.Building Nρ_inv cache in parallel...
2.Calculating nullity for each vertex in parallel...
3.Storing all vertices information...
Done.
----------Binding Network Summary:-------------
Number of species (n): 10
Number of conserved quantities (d): 5
Number of reactions (r): 5
L matrix: [1 0 0 0 0 1 0 1 2 2; 0 1 0 0 0 0 1 1 1 2; 0 0 1 0 0 0 0 0 0 0; 0 0 0 1 0 0 0 0 0 0; 0 0 0 0 1 1 1 2 2 3]
N matrix: [1 0 0 0 1 -1 0 0 0 0; 0 1 0 0 1 0 -1 0 0 0; 0 0 0 0 0 1 1 -1 0 0; 1 0 0 0 0 0 0 1 -1 0; 0 0 0 0 0 0 1 0 1 -1]
Direction of binding reactions: backward
Catalysis involved: No
Regimes constructed: Yes
Number of regimes: 79
  - Invertible + Asymptotic: 29
  - Singular +  Asymptotic: 22
  - Invertible +  Non-Asymptotic: 17
  - Singular +  Non-Asymptotic: 1

In [7]:
get_H(model,1)

10×10 SparseArrays.SparseMatrixCSC{Float64, Int64} with 32 stored entries:
 1.0   ⋅    ⋅    ⋅    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅    ⋅   1.0   ⋅    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅    ⋅    ⋅   1.0   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  ⋅    ⋅    ⋅    ⋅   1.0    ⋅     ⋅     ⋅     ⋅     ⋅ 
 1.0   ⋅    ⋅    ⋅   1.0  -1.0    ⋅     ⋅     ⋅     ⋅ 
  ⋅   1.0   ⋅    ⋅   1.0    ⋅   -1.0    ⋅     ⋅     ⋅ 
 1.0  1.0   ⋅    ⋅   2.0  -1.0  -1.0  -1.0    ⋅     ⋅ 
 2.0  1.0   ⋅    ⋅   2.0  -1.0  -1.0  -1.0  -1.0    ⋅ 
 2.0  2.0   ⋅    ⋅   3.0  -1.0  -2.0  -1.0  -1.0  -1.0

In [8]:
get_perm(model,10)

5-element Vector{Int8}:
  6
 10
  3
  4
  5

In [9]:
get_nullity(model,20)

0

In [ ]:
using LinearAlgebra
using SparseArrays
using Random
using Distributions

function calc_volume(
    Cs::AbstractVector{<:AbstractMatrix{<:Real}},
    C0s::AbstractVector{<:AbstractVector{<:Real}};
    confidence_level::Float64 = 0.95,
    contain_overlap::Bool = false,
    batch_size::Int = 100_000,
    log_lower::Float64 = -6.0,
    log_upper::Float64 = 6.0,
    tol::Float64 = 0.0,
    rel_tol::Float64 = 0.005,
    time_limit::Float64 = 40.0,
)::Vector{Tuple{Float64, Float64}}

    @assert length(Cs) == length(C0s) "Cs and C0s must have same length"
    n_regimes = length(Cs)
    @info "Number of polyhedra to calc volume: $n_regimes"
    n_regimes == 0 && return Tuple{Float64,Float64}[]

    # Dimensions & sanity
    n_dim = size(Cs[1], 2)
    for i in 1:n_regimes
        @assert size(Cs[i], 2) == n_dim "All Cs must have same column dimension"
        @assert size(Cs[i], 1) == length(C0s[i]) "size(Cs[$i],1) must match length(C0s[$i])"
    end

    # Wilson interval parameter
    z = quantile(Normal(), (1 + confidence_level) / 2)

    @inline function wilson_center_margin(count::Int, N::Int)
        count == 0 && return 0.0, 0.0
        P̂ = count / N
        denom = 1 + z^2 / N
        center = (P̂ + z^2 / (2N)) / denom
        margin = (z / denom) * sqrt(P̂ * (1 - P̂) / N + z^2 / (4N^2))
        return center, margin
    end

    # Global stats
    total_counts = zeros(Int, n_regimes)
    total_N = 0
    stats = fill((0.0, 0.0), n_regimes)
    rel_errors = fill(Inf, n_regimes)

    active_ids = collect(1:n_regimes)

    # Thread-local slot count (important: maxthreadid, not nthreads)
    n_slots = Threads.maxthreadid()
    thread_counts = [zeros(Int, n_regimes) for _ in 1:n_slots]

    # Thread-local RNG + x workspace
    # Use a stable seed per thread to avoid contention and keep reproducibility-ish.
    thread_rng = [MersenneTwister(0x12345678 + tid) for tid in 1:n_slots]
    thread_x = [Vector{Float64}(undef, n_dim) for _ in 1:n_slots]

    # Thread-local y workspaces: one vector per regime (length = m_i)
    # Use Float64 for speed; if your b is Float64 this is perfect.
    thread_y = [
        [Vector{Float64}(undef, size(Cs[i], 1)) for i in 1:n_regimes]
        for _ in 1:n_slots
    ]

    # Pre-grab b as Float64 vectors if possible (avoids repeated Real->Float64 conversions)
    # If b is already Vector{Float64}, this is just a cheap reference.
    b64 = Vector{Vector{Float64}}(undef, n_regimes)
    for i in 1:n_regimes
        bi = C0s[i]
        if bi isa Vector{Float64}
            b64[i] = bi
        else
            b64[i] = Float64.(bi)
        end
    end

    start_time = time()
    width = log_upper - log_lower

    while true
        (time() - start_time > time_limit) && break
        isempty(active_ids) && break

        # Monte Carlo batch
        Threads.@threads for _ in 1:batch_size
            tid = Threads.threadid()
            rng = thread_rng[tid]
            x = thread_x[tid]
            local_counts = thread_counts[tid]
            ywork = thread_y[tid]

            # x ~ Uniform(log_lower, log_upper)^n_dim
            @inbounds @simd for k in 1:n_dim
                x[k] = log_lower + width * rand(rng)
            end

            # test regimes
            for idx in active_ids
                A = Cs[idx]
                b = b64[idx]
                y = ywork[idx]

                # y = A*x  (sparse gemv)
                mul!(y, A, x)

                # check y + b >= -tol  (fuse add+check)
                ok = true
                @inbounds for k in 1:length(y)
                    if y[k] + b[k] < -tol
                        ok = false
                        break
                    end
                end
                ok || continue

                local_counts[idx] += 1
                contain_overlap || break
            end
        end

        # Reduce and reset counts; update N
        for c in thread_counts
            @inbounds for i in active_ids
                total_counts[i] += c[i]
                c[i] = 0
            end
        end
        total_N += batch_size

        # Update CI and prune active_ids
        new_active = Int[]
        sizehint!(new_active, length(active_ids))
        for i in active_ids
            center, margin = wilson_center_margin(total_counts[i], total_N)
            stats[i] = (center, margin)
            re = (center == 0.0) ? Inf : (margin / center)
            rel_errors[i] = re
            if re > rel_tol
                push!(new_active, i)
            end
        end
        active_ids = new_active
    end

    return stats
end

calc_volume (generic function with 2 methods)

In [10]:
poly = get_polyhedron(model,13)

Polyhedron CDDLib.Polyhedron{Float64}:
1-element iterator of Polyhedra.HyperPlane{Float64, Vector{Float64}}:
 HyperPlane([1.0, -1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], 0.0),
9-element iterator of Polyhedra.HalfSpace{Float64, Vector{Float64}}:
 HalfSpace([-1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0], 0.0)
 HalfSpace([-1.0, -0.0, -0.0, -0.0, -1.0, -0.0, 1.0, 1.0, -0.0, -0.0], -5.551115123125783e-17)
 HalfSpace([-1.0, -0.0, -0.0, -0.0, -1.9999999999999991, 0.9999999999999996, 0.9999999999999998, 0.9999999999999996, -0.0, -0.0], -1.4387158324070395e-16)
 HalfSpace([-1.0, -0.0, -0.0, -0.0, -0.9999999999999996, 0.9999999999999996, -0.0, 0.9999999999999996, -0.0, -0.0], -8.836043200944616e-17)
 HalfSpace([-0.0, -0.0, -0.0, -0.0, -6.6438561897747235, 3.3219280948873604, 3.3219280948873635, 3.3219280948873604, -3.3219280948873604, -0.0], -1.0)
 HalfSpace([-0.0, -0.0, -0.0, -0.0, -3.3219280948873613, 3.3219280948873613, -0.0, 3.3219280948873613, -3.3219280948873613, -

In [11]:
get_nullity(poly)

1

In [12]:
get_one_inner_point(model,15)

10-element Vector{Float64}:
 -3.2285887600360845
 -1.8363660260974508
  1.0401487381273555
  1.0799921908667722
 -0.08673402054159406
  0.3826340493141929
 -1.7454736472262322
 -2.416400084611071
 -6.4248849383704485
 -0.2580911843442892

In [13]:
vtx = get_vertex(model,16)

----------------Start calculating vertices neighbor graph, It may takes a while.----------------
Done.



BindingAndCatalysis.Vertex{Float64, Int8}(Bnc{Int8}([1 0 … 0 0; 0 1 … 0 0; … ; 1 0 … -1 0; 0 0 … 1 -1], [1 0 … 2 2; 0 1 … 1 2; … ; 0 0 … 0 0; 0 0 … 2 3], 5, 10, 5, Symbolics.Num[x₁, x₂, x₃, x₄, x₅, x₆, x₇, x₈, x₉, x₁₀], Symbolics.Num[q₁, q₂, q₃, q₄, q₅], Symbolics.Num[K₁, K₂, K₃, K₄, K₅], nothing, Vector{Int8}[[1, 2, 3, 4, 5], [1, 7, 3, 4, 5], [1, 8, 3, 4, 5], [1, 9, 3, 4, 5], [1, 10, 3, 4, 5], [6, 2, 3, 4, 5], [6, 7, 3, 4, 5], [6, 8, 3, 4, 5], [6, 9, 3, 4, 5], [6, 10, 3, 4, 5]  …  [1, 10, 3, 4, 10], [6, 2, 3, 4, 10], [6, 7, 3, 4, 10], [6, 10, 3, 4, 10], [9, 2, 3, 4, 10], [9, 7, 3, 4, 10], [9, 10, 3, 4, 10], [10, 2, 3, 4, 10], [10, 7, 3, 4, 10], [10, 10, 3, 4, 10]], Dict{Vector{Int8}, Int64}([6, 10, 3, 4, 6] => 31, [9, 2, 3, 4, 7] => 38, [1, 10, 3, 4, 9] => 63, [9, 10, 3, 4, 5] => 18, [9, 7, 3, 4, 9] => 65, [1, 2, 3, 4, 5] => 1, [9, 9, 3, 4, 9] => 66, [6, 2, 3, 4, 5] => 6, [10, 7, 3, 4, 7] => 41, [10, 10, 3, 4, 8] => 59…), Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 0, 0, 0, 0, 0, 0, 1, 0

In [14]:
vtxs = get_vertices(model,singular=false,asymptotic=true)[1:5]

5-element Vector{Vector{Int8}}:
 [1, 2, 3, 4, 5]
 [1, 7, 3, 4, 5]
 [1, 8, 3, 4, 5]
 [1, 9, 3, 4, 5]
 [1, 10, 3, 4, 5]

In [15]:
Cs = vtxs .|> x->get_C(model,x)
C0s = vtxs .|> x->get_C0(model,x)

5-element Vector{Vector{Float64}}:
 [0.0, 0.0, -0.3010299956639812, -0.3010299956639812, 0.0, 0.0, 0.0, -0.3010299956639812, 0.0, 0.0, -0.3010299956639812, -0.3010299956639812, -0.47712125471966244]
 [0.0, 0.0, -0.3010299956639812, -0.3010299956639812, 0.0, 0.0, 0.0, -0.3010299956639812, 0.0, 0.0, -0.3010299956639812, -0.3010299956639812, -0.47712125471966244]
 [0.0, 0.0, -0.3010299956639812, -0.3010299956639812, 0.0, 0.0, 0.0, -0.3010299956639812, 0.0, 0.0, -0.3010299956639812, -0.3010299956639812, -0.47712125471966244]
 [0.0, 0.0, -0.3010299956639812, -0.3010299956639812, 0.0, 0.0, 0.0, -0.3010299956639812, 0.0, 0.0, -0.3010299956639812, -0.3010299956639812, -0.47712125471966244]
 [0.0, 0.1505149978319906, -0.1505149978319906, 0.0, 0.1505149978319906, 0.1505149978319906, 0.1505149978319906, 0.1505149978319906, 0.0, 0.1505149978319906, -0.1505149978319906, -0.1505149978319906, -0.17609125905568124]

In [24]:
BindingAndCatalysis.calc_volume(Cs, C0s)

[ Info: Number of polyhedra to calc volume: 5
[ Info: All regimes converged after 63100000 samples.
[ Info: Total samples: 63100000, Elapsed: 27.59 s


5-element Vector{Tuple{Float64, Float64}}:
 (0.11675206019264908, 0.0005745520462504446)
 (0.03699819465059229, 0.00018497815746441843)
 (0.0053002387670851685, 2.6472290800560867e-5)
 (0.018409006099020915, 9.200686029142326e-5)
 (0.0024307117494759313, 1.214983314273485e-5)

In [29]:
calc_volume(Cs, C0s)

5-element Vector{Tuple{Float64, Float64}}:
 (0.11674622687798945, 0.0005745395899577155)
 (0.03715644449861473, 0.00018535810331844069)
 (0.005319267371377215, 2.6565515126118516e-5)
 (0.018338174733020157, 9.12781130755793e-5)
 (0.0024254416362686107, 1.212708117764716e-5)

In [13]:
vtx_grh = BindingAndCatalysis.get_vertices_graph!(model,full=false)

BindingAndCatalysis.VertexGraph{Int8}(Graphs.SimpleGraphs.SimpleGraph{Int64}(349, [[2, 3, 4, 5, 6, 11, 14, 19, 22, 32, 42, 60, 68], [1, 3, 4, 5, 7, 12, 15, 20, 23, 33, 43, 61, 69], [1, 2, 4, 5, 8, 13, 16, 24, 44], [1, 2, 3, 5, 9, 17, 25, 62], [1, 2, 3, 4, 10, 18, 21, 26, 45, 63, 70], [1, 7, 8, 9, 10, 11, 14, 19, 27, 34, 46, 71], [2, 6, 8, 9, 10, 12, 15, 20, 28, 35, 47, 72], [3, 6, 7, 9, 10, 13, 16, 29, 48], [4, 6, 7, 8, 10, 17, 30], [5, 6, 7, 8, 9, 18, 21, 31, 49, 73]  …  [5, 26, 45, 63, 68, 69, 73, 76, 79], [6, 27, 34, 46, 68, 72, 73, 74, 77], [7, 28, 35, 47, 69, 71, 73, 75, 78], [10, 31, 49, 70, 71, 72, 76, 79], [14, 38, 53, 64, 68, 71, 75, 76, 77], [15, 39, 54, 65, 69, 72, 74, 76, 78], [18, 56, 67, 70, 73, 74, 75, 79], [19, 40, 57, 68, 71, 74, 78, 79], [20, 41, 58, 69, 72, 75, 77, 79], [21, 59, 70, 73, 76, 77, 78]]), Vector{BindingAndCatalysis.VertexEdge{Int8}}[[BindingAndCatalysis.VertexEdge{Int8}(22, 5, sparsevec(Int8[5, 6], Int8[-1, 1], 10), nothing), BindingAndCatalysis.VertexEd

In [12]:
@show Threads.nthreads() Threads.maxthreadid() Threads.threadid()

Threads.nthreads() = 24
Threads.maxthreadid() = 49
Threads.threadid() = 1


1